In [45]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# reading full data
pd.read_csv("Data/bank-additional-full.csv")

In [ ]:
pd.read_csv("Data/bank.csv")

In [ ]:
train_df = pd.read_csv("Data/bank-additional-full.csv", sep=';')
train_df.head()

In [ ]:
test_df = pd.read_csv("Data/bank-additional.csv", sep=';')
test_df.head()

In [ ]:
train_df.describe()

In [ ]:
test_df.describe()

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

Exploratory Data Analysis

In [ ]:
train_df["y"].value_counts(normalize = True)

In [ ]:
test_df["y"].value_counts(normalize = True)

Changing month value into numerical values

In [58]:
month_to_nv = {
    "jan": 1,
    "feb": 2,
    "mar": 3,
    "apr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "aug": 8,
    "sep": 9,
    "oct": 10,
    "nov": 11,
    "dec": 12
}

for df in [train_df, test_df]:
    if 'month' in df.columns and df['month'].dtype == 'object':
        df['month'] = df['month'].map(month_to_nv)

In [ ]:
test_df

In [ ]:
train_df

In [ ]:
# Changing Yes and No to 1 and 0 (Where Yes is 1 and No is 0)
yes_no_columns = train_df.columns[train_df.isin(['yes', 'no']).any()]
train_df[yes_no_columns] = train_df[yes_no_columns].applymap(lambda x: 1 if x == 'yes' else (0 if x == 'no' else x))

In [ ]:
train_df.head()

In [ ]:
yes_no_columns = test_df.columns[test_df.isin(['yes', 'no']).any()]
test_df[yes_no_columns] = test_df[yes_no_columns].applymap(lambda x: 1 if x == 'yes' else (0 if x == 'no' else x))

In [ ]:
test_df

In [ ]:
P_values = ['age', 'default', 'housing', 'loan', 'day_of_week', 'month', 'duration', 'campaign', 'previous']
plt.figure(figsize=(15, 20))
for i, P_values in enumerate(P_values):
    plt.subplot(5, 2, i + 1)
    if train_df[P_values].dtype == 'object':
        sns.stripplot(x = P_values, y='y', data = train_df, jitter=True, alpha=0.6)
    else:
        plt.scatter(train_df[P_values], train_df['y'], alpha = 0.5, color = "red")
        plt.xlabel(P_values)
        plt.ylabel('y')
        plt.title(f'{P_values} vs y')

plt.tight_layout()
plt.show()

Converting Columns into Binary

In [38]:
for column in train_df.select_dtypes(include = "object").columns:
    if len(train_df[column].unique()) == 2 and column != 'month':
        train_df[column] = train_df[column].apply(lambda x: 1 if x == "yes" else 0)

Encoding Columns

In [39]:
obj_columns = train_df.select_dtypes(include = "object").columns
train_df = pd.get_dummies(train_df, columns = obj_columns)
train_df[train_df.select_dtypes(include = "bool").columns] = train_df[train_df.select_dtypes(include = "bool").columns].astype(int)

Feature Extraction (Numerical and Categorical Extraction)

In [ ]:
numerical_feature = []
categorical_feature = []

for column in train_df.columns:
    if column == "Target":
        continue
    elif len(train_df[column].unique()) <= 10:
        categorical_feature.append(column)
    else:
        numerical_feature.append(column)

Cleraing Outliers

In [ ]:
no_o_train_df = train_df.copy()
for column in numerical_feature:
    IQR = no_o_train_df[column].quantile(0.75) - no_o_train_df[column].quantile(0.25)
    lower_bound = no_o_train_df[column].quantile(0.25) - 1.5 * IQR
    upper_bound = no_o_train_df[column].quantile(0.75) + 1.5 * IQR
    no_o_train_df = no_o_train_df[(no_o_train_df[column] >= lower_bound) & (no_o_train_df[column] <= upper_bound)]

In [ ]:
plt.figure(figsize = (12, 8))
sns.histplot(data = no_o_train_df["age"], kde=True, bins=20)

plt.show()

In [ ]:
train_df_m = no_o_train_df[numerical_feature + ["y"]] \
    .melt(id_vars = "y", value_vars = numerical_feature,
         var_name = "feature_name", value_name = "feature_value")

In [ ]:
train_df.head()

In [ ]:
sns.catplot(kind='box', data = train_df_m,
           x='y', y='feature_value',
           col='feature_name', col_wrap=3, sharey=False)

plt.show()

In [65]:
X_train = train_df.drop('y', axis=1)
y_train = train_df['y']
X_test = test_df.drop('y', axis=1)
y_test = test_df['y']

In [66]:
X_all = pd.concat([X_train, X_test], keys=["train", "test"])
X_all = pd.get_dummies(X_all, drop_first=True)

In [67]:
X_train = X_all.loc["train"]
X_test = X_all.loc["test"]

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

In [ ]:
model.predict(X_test,)

In [ ]:
y_test

In [ ]:
model.score(X_test, y_test)

In [ ]:

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("\n", Counter(y_test), "\n", Counter(y_pred))